This is based on code from:

https://huggingface.co/spaces/facebook/incoder-demo/blob/main/modules/app.py

## Prerequisites

1. Compile the Rust code in `../rs` with `cargo build`.
2. You will need a GPU with 10+ GB of memory.

In [1]:
from pathlib import Path
import re
import model
import main
import importlib
import torch

## Choose the Model

If you reload the next cell, be sure to reload the TypeInference class in the next cell.

In [2]:
m = model.init_model("facebook/incoder-1B")

## Load Type Inference

It is a huge pain to reload the whole notebook when we modify the TypeInference class. 
Reloading it is straightforward with importlib. However, we also have to clear tensors in VRAM that
were placed by an earlier invocation.

Set the `temperature > 0.0` in the constructor to sample types instead of producing types greedily.
However, note that greedy sampling is substantially faster.

In [5]:
importlib.reload(main)
typeinf = main.TypeInference(**m)
torch.cuda.empty_cache()

## Run Inference

Inference reads a file and produces its contents with types filled in.

In [9]:
print(typeinf.infer(Path("../samples/joydeep_rrt.js")))

0it [00:00, ?it/s]

let rrt = require('rrt');
let Point = rrt.Point;
let Line = rrt.Line;
let c = lib220.newCanvas(400, 400);

function drawMap(canvas: HTMLCanvasElement,  map: Map) {
  map.forEach(function(l: Line) {
    drawLine(canvas, l.p1, l.p2, [0, 0, 0]);
  });
}

function drawPoint(canvas: HTMLCanvasElement,  p: Point,  color: string) {
  canvas.drawFilledCircle(p.x, p.y, 2, color);
}

function drawLine(canvas: HTMLCanvasElement,  p1: Point,  p2: Point,  color: Color) {
  canvas.drawLine(p1.x, p1.y, p2.x, p2.y, color);
}

function drawTree(canvas: HTMLCanvasElement,  tree: Tree) {
  let p1 = tree.node;
  tree.children.forEach(function(p2: P2) {
    canvas.drawLine(p1.x, p1.y, p2.node.x, p2.node.y, [0, 0, 1]);
    drawTree(canvas, p2);
  });
}

function drawPath(canvas: HTMLCanvasElement,  path: Path,  color: string) {
  for (let i = 1; i < path.length; ++i) {
    drawLine(c, path[i], path[i - 1], color);
  }
}

class Tree {
  constructor(node, parent) {
    this.node = node;
    this.parent = pare

To see a succinct summary of the types that were inserted, see the type_log in the `typeinf` object.

In [10]:
", ".join([ item["type"] for item in typeinf.type_log ])

'HTMLCanvasElement, Map, Line, HTMLCanvasElement, Point, string, HTMLCanvasElement, Point, Point, Color, HTMLCanvasElement, Tree, P2, HTMLCanvasElement, Path, string, Vector, Vector, number, number, number, Map, Point, Point, any, any, number, number, Map, PlanOptions, Point, Point, Point, Point, number, number, number, Point, number, Point, number, number, number, Path, Map, Array<string>, Array<string>, Array<string>, Map<string, any>, Function, Function, number, number, number, number'

The log also has the prompts that were sent to the model.

In [ ]:
for item in typeinf.type_log:
    print("Type = ", item["type"], "prompt = ", item["prompt"])
    print("*********************")